In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

#### Load and prepare the MNIST dataset.

In [24]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [25]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28, 1), (60000,), (10000, 28, 28, 1), (10000,))

#### Use `tf.data` to batch and shuffle the dataset:

In [26]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [27]:
train_ds

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>

#### Build the `tf.keras` model using the Keras model subclassing API:
-https://www.tensorflow.org/guide/keras#model_subclassing

In [28]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
# Create an instance of the model
model = MyModel()

#### Choose an optimizer and loss function for training:

In [29]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

#### Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [30]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

#### Use `tf.GradientTape` to train the model:

In [31]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
    train_loss(loss)
    train_accuracy(labels, predictions)

#### Test the model:

In [32]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [23]:
# 1st try
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
    template = "Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}"
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))

W0113 14:33:24.280681 140477920395392 base_layer.py:1790] Layer my_model_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Epoch 1, Loss: 0.13873542845249176, Accuracy: 95.80833435058594, Test Loss: 0.06219889596104622, Test Accuracy: 98.0999984741211
Epoch 2, Loss: 0.04351449012756348, Accuracy: 98.62333679199219, Test Loss: 0.05381500720977783, Test Accuracy: 98.20999908447266
Epoch 3, Loss: 0.02358250506222248, Accuracy: 99.24166870117188, Test Loss: 0.0502362959086895, Test Accuracy: 98.40999603271484
Epoch 4, Loss: 0.01344351377338171, Accuracy: 99.55333709716797, Test Loss: 0.05876682698726654, Test Accuracy: 98.4000015258789
Epoch 5, Loss: 0.00959051214158535, Accuracy: 99.6933364868164, Test Loss: 0.06192786991596222, Test Accuracy: 98.41999816894531


In [33]:
# 2nd try
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
    template = "Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}"
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))

W0113 14:38:19.747588 140477920395392 base_layer.py:1790] Layer my_model_2 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Epoch 1, Loss: 0.13467423617839813, Accuracy: 96.04166412353516, Test Loss: 0.05873800069093704, Test Accuracy: 98.02999877929688
Epoch 2, Loss: 0.042097821831703186, Accuracy: 98.65666961669922, Test Loss: 0.04994458332657814, Test Accuracy: 98.43000030517578
Epoch 3, Loss: 0.022904707118868828, Accuracy: 99.2933349609375, Test Loss: 0.06059320643544197, Test Accuracy: 98.04999542236328
Epoch 4, Loss: 0.0133378179743886, Accuracy: 99.5250015258789, Test Loss: 0.058298103511333466, Test Accuracy: 98.3699951171875
Epoch 5, Loss: 0.008877982385456562, Accuracy: 99.7300033569336, Test Loss: 0.061415255069732666, Test Accuracy: 98.37999725341797
